In [5]:
import pandas as pd


In [7]:
!pip install wikipedia
!pip install requests
!pip install BeautilfulSoup
!pip install time


  Could not find a version that satisfies the requirement BeautilfulSoup (from versions: )
No matching distribution found for BeautilfulSoup


  Could not find a version that satisfies the requirement time (from versions: )
No matching distribution found for time


In [1]:
!pip install lxml

In [1]:
import requests
from bs4 import BeautifulSoup
import bs4
import time
import numpy as np
import re
import time
#the goal is scrapping wikipedia in order to obtain information about citrus fruit
#We take the description section of each one and then we will extract main info.


In [88]:
page_citrus = 'https://en.wikipedia.org/wiki/List_of_citrus_fruits'
page_salads = 'https://en.wikipedia.org/wiki/List_of_leaf_vegetables'
page_spices = 'https://en.wikipedia.org/wiki/List_of_culinary_herbs_and_spices'

#A function that gets the URL of the page to be scraped
#,gets the html content and uses BeautifulSoup to parse html content

def make_soup(link):
    get_page = requests.get(link)
    html = get_page.content
    soup = BeautifulSoup(html, 'html.parser')
    return  soup


#####This function create a list with all the link of the foods in a wikipedia Page.
def make_link_list(wiki_page_to_scrap):
    start_time = time.time()
    link_table = []
    soup = make_soup(wiki_page_to_scrap)
    table = soup.find('table',{'class':'wikitable sortable'})
    
            ### This first loop is used to scrap Wiki table Data.
    if isinstance(table , bs4.element.Tag):        
    
        table_cells = []
        table = soup.find('table',{'class':'wikitable sortable'})
        for row in table.find_all("tr"):
            cells = row.find_all(['th' , 'td'])
            table_cells.append(cells)
        
        ### This loop is used to locate the "Common name" column index in our table cells
        indices = []
        for j in table_cells:
            for i, elem in enumerate(j):
                elem = str(elem)
                if 'Common' in elem:
                    indices.append(i)
        indice = indices[0]
        
        
        ### Here we implement a loop to keep only the string of the Common name column.
        
        for cell in table_cells[1:] :
            link_table.append(cell[indice].text)
            
        ### Let's clean the table.
        for link in range(len(link_table)):
            link_table[link] = link_table[link].strip('\n')
            
        print('cpu time for the table schema = {:.4f} sec.'.format(time.time() - start_time))
            
            ### Here is when the Wiki page is just an Alphabetical List.    
    else:       
        for row in soup.find_all('div' , {'class':'div-col'}):
            for col in row.find_all('li'):
                species = col.text
                ###We just keep the common name of the species, because only the common name is used in recipes.
                only_common_species = re.sub("[\(\[].*?[\)\]]", "", species) 
                only_common_species , sep , tail = only_common_species.partition(',')
                link_table.append(only_common_species)
                
                ###Cleaning of the list, we remove all the occurence of string begining by List.
        for word in link_table[:]:
            if word.find('List') != -1:
                link_table.remove(word)
            
        print('cpu time for the list schema = {:.4f} sec.'.format(time.time() - start_time))     

    return link_table  
    
#list_salad = make_link_list(page_salads)

list_citrus = make_link_list(page_citrus)
print('Number of citrus' , list_citrus)

cpu time for the table schema = 0.2433 sec.
Number of citrus ['Amanatsu', 'Balady citronIsrael citron', 'Bergamot orange', 'Bitter orangeSeville orangeSour orangeBigarade orangeMarmalade orange', 'Blood orange', "Buddha's handBushukanFingered citron", 'CalamondinCalamansi', 'Cam sành', 'Citron', 'Clementine', 'Corsican citron', 'Desert lime', 'Etrog', 'Finger lime', 'Florentine citron', 'Grapefruit', 'Greek citron', 'HyuganatsuKonatsuTosakonatsuNew Summer Orange', 'First LadyAnadomikan', 'Kabosu', 'Kaffir lime', 'Key lime', 'Kinnow', 'Kiyomi', 'Kumquat', 'Lemon', 'Lime', 'Mandarin orangeMandarinMandarine', 'Mangshanyegan', 'Meyer lemon', 'Moroccan citron', 'Myrtle-leaved orange tree', 'OrangeSweet orange', 'OroblancoSweetie', 'Papeda', 'Persian limeTahiti limeBearss lime', 'PomeloPummeloPommeloShaddock', 'Ponderosa lemon', 'RangpurLemandarin', 'Round limeAustralian limeAustralian round lime', 'SatsumaCold hardy mandarinSatsuma mandarinSatsuma orangeChristmas orangeTangerine', 'Shangjua

In [29]:
list_citrus = make_link_list(page_citrus)
print('Number of citrus' , len(list_citrus))

#list_salad = make_link_list(page_salads) 
#print('Number of salads' , len(list_salad))
### Attention latin name were scrapped => tackle the traduction between latin & common name
## maybe change in the "make_link_list"

#list_spices = make_link_list(page_spices)
#print('Number of spices' , len(list_spices))


cpu time for the table schema = 0.5778 sec.
Number of citrus 57


In [24]:
# Let's create a dictionary with the different categories of vegetables. 
#Categories will be: 
    #Stem Vegetables
    #Leaves Vegetables
    #Flowers Vegetables
    #Bulb Vegetables
    #Beans
    #Roots Vegetables
    #Tubers Vegetables
    #Fruits Vegetables
    #Fungi Vegetables


vegetable_dict = {'Stem' : 0 , 'Leaves' : 0 , 'Flowers' : 0 , 'Bulb' : 0 , 'Beans' : 0 , 'Roots' : 0 , 'Tubers' : 0 , 'Fruits' : 0 , 'Fungi': 0 }


In [58]:
list_citrus = make_link_list(page_citrus)

<table class="wikitable sortable">
<tbody><tr>
<th style="width:20%;">Common<br/>name(s)
</th>
<th>Image
</th>
<th>Taxonomic<br/>name/constituents
</th>
<th>Notes
</th></tr>
<tr>
<td><a href="/wiki/Amanatsu" title="Amanatsu">Amanatsu</a>
</td>
<td><a class="image" href="/wiki/File:Citrus_kawanonatsudaidai_fruit.jpg"><img alt="Citrus kawanonatsudaidai fruit.jpg" data-file-height="2592" data-file-width="3872" decoding="async" height="80" src="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Citrus_kawanonatsudaidai_fruit.jpg/120px-Citrus_kawanonatsudaidai_fruit.jpg" srcset="//upload.wikimedia.org/wikipedia/commons/thumb/d/df/Citrus_kawanonatsudaidai_fruit.jpg/180px-Citrus_kawanonatsudaidai_fruit.jpg 1.5x, //upload.wikimedia.org/wikipedia/commons/thumb/d/df/Citrus_kawanonatsudaidai_fruit.jpg/240px-Citrus_kawanonatsudaidai_fruit.jpg 2x" width="120"/></a>
</td>
<td><i>Citrus natsudaidai</i>
</td>
<td>Yellowish-orange in colour, about the size of <a href="/wiki/Grapefruit" title="Grapefru